<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:250%;text-align:center;border-radius:40px 40px;">♻️Waste classification(ResNet50)♻️</p>


<img src="https://i.ibb.co/bdWGLcj/Screenshot-2021-12-14-232222.png"> 
<img src="https://i.ibb.co/C1CWmzL/Screenshot-2021-12-14-232416.png">  

<p style="background-color:#CABFC1;font-family:newtimeroman;font-size:200%;text-align:left;"> 📌This notebook is beginner friendly and contains an end to end deployed project.
</p>

*  **https://github.com/raj-gupta1/Automatic-Waste-Classification-using-Deep-Learning-ResNet50-architecture** 

* **Architecture used**<br>
    1. ResNet50
    2. No. of classes: 2
    3. Classes: Organic and Recyclable waste
    <br><br>

<p style="background-color:#CABFC1;font-family:newtimeroman;font-size:200%;text-align:left;"> 📌Tools for project deployment.
</p>

* **Libraries used**<br>
    1. Flask==1.1.2
    2. numpy==1.18.4
    3. tensorflow==2.2.0
    4. Werkzeug==1.0.1
    <br><br>

* **Tools used**<br>
    1. Pycharm
    2. Google Colab
    3. git
    <br><br>

<a id="1"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Importing the library</p>

In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [2]:
import os
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Splitting the data into train and validate</p>


# Enter Image size and shape (preferrably use the image shape based upon the paper you are using.)

In [3]:
IMAGE_SIZE = [224, 224]

train_path = "/kaggle/input/waste-classification-data/DATASET/TRAIN/"
valid_path = "/kaggle/input/waste-classification-data/DATASET/TEST/"

 
# <a id="1"></a> <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">ResNet50 Transfer Learning Model creation</p>

In [4]:
# Import the Resnet50 architecture
from tensorflow.keras.applications import ResNet50

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in resnet.layers:
    layer.trainable = False

# Finding the number of classes

In [6]:
folders = glob('/kaggle/input/waste-classification-data/DATASET/TRAIN/*')

In [7]:
x = Flatten()(resnet.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Data Augumentation</p>


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Creating training dataset</p>


In [13]:
training_set = train_datagen.flow_from_directory('/kaggle/input/waste-classification-data/DATASET/TRAIN',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


KeyboardInterrupt



<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Creating testing dataset</p>


In [ ]:
test_set = test_datagen.flow_from_directory('/kaggle/input/waste-classification-data/DATASET/TEST',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

# Creating a model checkpoint

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
MODEL_DIR = "/kaggle/input/waste-classification-data"

if not os.path.exists(MODEL_DIR):  #If the directory does not exist, create it.
    os.makedirs(MODEL_DIR)
checkpoint = ModelCheckpoint(filepath=os.path.join(MODEL_DIR, "model-{epoch:02d}.h5"), save_best_only=True) 

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Fitting the model</p>


# Training the model on 20 epochs

In [ ]:
#fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
#plt.savefig('LossVal_loss_eff')

# accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
#plt.savefig('AccVal_acc_eff')

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Saving the model</p>


In [ ]:
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from glob import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2


In [ ]:
from tensorflow.keras.models import load_model

model.save('resnet.h5')

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:150%;text-align:center;border-radius:40px 40px;">Predicting the model</p>


In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

# Loading the model to do predictions

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('resnet.h5')

# Checking the Predictions using validation dataset

In [ ]:
# test data to check model and predict 
img = image.load_img('../input/waste-classification-data/DATASET/TEST/O/O_12577.jpg', target_size = (224, 224))

In [ ]:
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
result = model.predict(img)

In [ ]:
training_set.class_indices

In [ ]:
if result[0][0] == 1:
    prediction = 'Recyclable'
else:
    prediction = 'Organic'

In [ ]:
prediction

<a id="6"></a>
# <p style="background-color:#615154;font-family:newtimeroman;color:#CABFC1;font-size:250%;text-align:center;border-radius:40px 40px;">Keep Learning and Keep Upvoting
</p>